<a href="https://colab.research.google.com/github/okapee/aidialog/blob/master/Bi_directional_LSTM%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%96%87%E7%AB%A0%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 概要

Bi-directional LSTMを用いてチャットボット用のロジックを実装しています。
ここで保存したモデルと処理ロジックをHerokuにデプロイしてLINE Botを動作させます。

## 実装

In [1]:
from keras.layers import Dense , Activation , Bidirectional, LSTM,Flatten
from keras.models import Sequential 
import numpy as np

In [2]:
from google.colab import files

In [3]:
file = files.upload()
!ls

Saving dialog_mini.txt to dialog_mini.txt
dialog_mini.txt  sample_data


In [5]:
FILE_PATH = "./dialog_mini.txt"
text=""
with open(FILE_PATH, 'r',encoding="utf-8") as f:
  for line in f:
    lines = line.split()
    text += " ".join(lines)
text = text.lower()
text

',,、,,する。うん。あー。昨日ー、なか、最後だったじゃん、バイトが。うん。でね、「人名1」君と,,固有名詞出してる。。「人名1」君と「人名2」と「人名3」君と,,。ちゃんが来て、で、「人名2」は、魚の皿を買ってって、で、るいちゃんは自由帳を「人名1」君に買ってもらってて、で、なんかね、そうそうそう、あの『施設名1』のさ、あのカラオケボックスあるじゃん。『施設名1』っていう。はあ、ああ、ああ、ああ。あの、「地名1」で行く、あのカラオケボックス。はいはいはい。オールするとこ。うんうんうん。あそこのー、店長と知り合いじゃん。知り合いなの。うん。知り合いなの。なんかね、『施設名1』の会員のやつ,,あーーーあ。に登録して、そしたらー、なんか、メール、ダイレクトメールが、きてて、で、なんか、店長は暇だったらしくて、あの“メル友になってください”って、あ、なんか、ちが、そういうのじゃなくて。そんなの。。ちゃ、わかんないけど。ちが、ちが、ちが。で、普通になんかdmが書いてあってー、あたしはいっつも、そんなにだって部活のときしか行かないからー、別にいつもは使ってないのにー、dmもらうのは悪いなーと思って、“すいませねー、いつも”みたいなこと返信してたら、なか、向こうから何かと返事が返ってきてて、で、まあ、それから適当になんか話をして、で、あたしが「店名1」で働いてるってことが発覚したの。あーあー。で、“ちかいじゃないすか、うち結構買いもの行ってますよ”っていう話んなって、でー、なんか、それからちょくちょく来てくれるようんなって、で、あたしが向こうに行ったときも、なか、サービスしてくれたりとか、したの。なんか、「学園祭名」んときに、「人名5」さんたちと、とか、「人名6」先輩とかと、あの行ったときにも、なか、お米、プレゼントの企画をやってて、なか、くじ引き,,カラオケで。そうそう。うん。くじ引きして、なんか当たったら、米をもらえるんだけど、1キロか2キロか忘れたな。1キロかな。うん。なんか、「人名5」さんそんときにー、食う米に困っててー、でー,,欲しかったんだ。本気で欲しかったらしくて、ほんとはでも、それ、なんか、券を持ってないとできなかったの。あーあーあーあー。でも、「人名7」店長が、“いいよ、いいよ、やってよ”とかいってやって、やらしてくれて、で、あたし、「人名5」さん以外の人は

In [6]:
chars = set(text)
nb_chars = len(chars)
char2index = dict((c,i) for i , c in enumerate(chars))
index2char = dict((i,c) for i , c in enumerate(chars))

In [7]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []

for i in range( 0,len(text) - SEQLEN , STEP ):
  input_chars.append(text[i:i+SEQLEN])  
  label_chars.append(text[i+SEQLEN])  
  
  

In [9]:
X = np.zeros((len(input_chars),SEQLEN,nb_chars),dtype=np.float32)
Y = np.zeros((len(input_chars),nb_chars),dtype=np.float32)
for i , input_char in enumerate(input_chars):
  for j , ch in enumerate(input_char):
    X[i,j,char2index[ch]] = 1
  Y[i,char2index[label_chars[i]]] = 1
    


In [10]:
X.shape

(8200, 10, 443)

In [11]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 50
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(Bidirectional( LSTM(HIDDEN_SIZE, input_shape=(SEQLEN , nb_chars))))
model.add( Dense ( nb_chars ))
model.add( Activation ( "softmax" )) 
model.compile( loss = "categorical_crossentropy" , optimizer = "rmsprop" )

for iteration in range(NUM_ITERATIONS):
  print("=" * 50)
  print("Iteration #: %d" % (iteration))
  model.fit(X,Y,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS_PER_ITERATION)
  test_index = np.random.randint(len(input_chars))
  test_chars = input_chars[test_index]
  print("Generating form sead: {}".format(test_chars))
  print(test_chars , end="")
  for i in range(NUM_PREDS_PER_EPOCH):
    Xtest = np.zeros((1,SEQLEN,nb_chars))
    for i , ch in enumerate(test_chars):
      Xtest[0,i,char2index[ch]]=1
    pred = model.predict(Xtest,verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred,end="")
    test_chars = test_chars[1:] + ypred
  print() 
   

Iteration #: 0
65/65 [==============================] - 1s 8ms/step - loss: 4.6119
Generating form sead: 「地名8」。「地名8
「地名8」。「地名8、、名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名、名名名
Iteration #: 1
65/65 [==============================] - 1s 9ms/step - loss: 4.2559
Generating form sead: そう、すっごい懐かし
そう、すっごい懐かし。。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」。。」
Iteration #: 2
65/65 [==============================] - 1s 8ms/step - loss: 4.1134
Generating form sead: 。あの、なんかねー、
。あの、なんかねー、、、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、あ、あ、、
Iteration #: 3
65/65 [==============================] - 1s 9ms/step - loss: 4.0124
Generating form sead: じゃん。知り合いなの
じゃん。知り合いなの、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ、あ
Iteration #: 4
65/65 [==============================] - 1s 9ms/step - loss: 3.9307
Generatin

## モデルの保存

In [12]:
import os
#resultsディレクトリを作成
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

# 作成したモデルを保存
model.save('Bi-directional_model.h5')

In [13]:
# result_dirをローカルに保存
from google.colab import files
files.download('/content/Bi-directional_model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>